In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from torchvision.transforms import Resize
import torchvision.transforms as transforms
import torch
import plenoptic as po
import scipy.io as sio
import os
import os.path as op
import einops
from typing import Union, Tuple, Callable, List, Dict, Optional
import glob
import math
import pyrtools as pt
from tqdm import tqdm
from PIL import Image
%load_ext autoreload
%autoreload 

# We need to download some additional images for this notebook. In order to do so,
# we use an optional dependency, pooch. If the following raises an ImportError or ModuleNotFoundError 
# then install pooch in your plenoptic environment and restart your kernel.
DATA_PATH = po.data.fetch_data('portilla_simoncelli_images.tar.gz')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# so that relative sizes of axes created by po.imshow and others look right
plt.rcParams['figure.dpi'] = 72

# set seed for reproducibility
po.tools.set_seed(1)

In [ ]:
# These variables control how long metamer synthesis runs for. The values present here will result in completed synthesis,
# but you may want to decrease these numbers if you're on a machine with limited resources.
short_synth_max_iter = 100
long_synth_max_iter = 300
longest_synth_max_iter = 400

# Portilla-Simoncelli Texture Metamer

In this tutorial we will aim to replicate [Portilla & Simoncelli (1999)](https://www.cns.nyu.edu/pub/eero/portilla99-reprint.pdf). The tutorial is broken into the following parts:

1. Introduce the concept of a Visual Texture.
2. How to synthesize metamers for the Portilla & Simoncelli texture model. 
3. Demonstrate the importance of different classes of statistics.
4. Extending the model to synthesizing color metamers


Note that this notebook takes a long time to run (roughly an hour with a GPU, several hours without), because of all the metamers that are synthesized.

## 1. What is a visual texture?

The simplest definition is a repeating visual pattern. Textures encompass a wide variety of images, including natural patterns such as bark or fur, artificial ones such as brick, and computer-generated ones such as the Julesz patterns ([Julesz 1978](https://link.springer.com/article/10.1007/BF00336998), [Yellot 1993](https://opg.optica.org/josaa/abstract.cfm?uri=josaa-10-5-777)). Below we load some examples.  

The Portilla-Simoncelli model was developed to measure the statistical properties of visual textures.  Metamer synthesis was used (and can be used) in conjunction with the Portilla-Simoncelli texture model to demonstrate the necessity of different properties of the visual texture.  We will use some of these example textures to demonstrate aspects of the Portilla Simoncelli model.

In [ ]:
# For the remainder of the notebook we will use this helper function to
# run synthesis so that the cells are a bit less busy.

# Be sure to run this cell.


def run_synthesis(img, model, loss_function:Callable[[torch.Tensor, torch.Tensor], torch.Tensor], im_init=None):
    r""" Performs synthesis with the full Portilla-Simoncelli model. 

        Parameters
        ----------
        img : Tensor
            A tensor containing an img.
        model :
            A model to constrain synthesis.
        im_init: Tensor
            A tensor to start image synthesis.

        Returns
        -------
        met: Metamer
            Metamer from the full Portilla-Simoncelli Model

        """
    if im_init is None:
        im_init = torch.rand_like(img) * .01 + img.mean()
    met = po.synth.MetamerCTF(img, model, loss_function=loss_function, initial_image=im_init,
                              coarse_to_fine='together')
    met.synthesize(
        max_iter=long_synth_max_iter, 
        store_progress=True,
        change_scale_criterion=None,
        ctf_iters_to_check=3, # todo: tinker with this
        )
    return met

## 2. Using the Portilla-Simoncelli Model to Compute Realistic Color Metamers

Before adapting the model to synthesize metamers better on color texture images, let's have a look at the existing model's behavior.

### Combining Channel Information in RGB-Images
- The current implementation of the Portilla-Simoncelli Model computes the statistics on all channels separately, i.e., treats each channel as a separate image, resulting in unrealistic metamers (see demo below).
- This is why we will play around with color transforms in the forward method.
- We will first mmic what [Brown 2023](https://dl.acm.org/doi/full/10.1145/3564605#sec-supp) et al. did.
- In particular, we will write our custom Portilla-Simoncelli class, where we do the following steps:

#### Implementation:
0. **Transform to LMS**:
    - transform RGB to an LMS-like space, then from that into opposing color channels (define rgb2opc)
1. **Channel Separation**: The input image is split into its constituent channels.
    - compute L2 loss separately on each channel, the use torch.logsumexp to comine them
2. **Independent Processing**: Each channel is processed independently to compute the relevant statistics.
    - then add cross-channel correlations as in Brown
3. **Normalization and Weighting**: The computed statistics are normalized and weighted to ensure they contribute appropriately to the final representation. E.g., the green channel might be assigned higher weights as it  is often more sensitive in human vision.
4. **Fusion and Integration**: The normalized and weighted statistics are combined to form the final multi-channel representation.

### Synthesis on Color-Texture Images
#### 1. Synthesis with Original Model

In [ ]:
path_to_unsplash = '../../../ceph/Datasets/select_color_textures_unsplash'

In [ ]:
# import select color images

# Step 1: Check if the path to the images exists
# You can specify the extensions of the images you want to load, e.g., *.jpg, *.png, etc.
image_files = glob.glob(os.path.join(path_to_unsplash, '*.jpg'))  # Assuming images are in .jpg format
image_files.extend(glob.glob(os.path.join(path_to_unsplash, '*.png')))  # Include .png format as well

# Step 2: Load the images
images = []
for image_file in image_files:
    try:
        img = Image.open(image_file)
        images.append(img)
    except Exception as e:
        print(f"Error loading image {image_file}: {e}")
#po.imshow(po.tools.load_images(path_to_unsplash, as_gray=False), as_rgb=True)


In [ ]:
# transform the images to tensors and then resize them to 256x256
#images = [torch.tensor(np.array(img)).permute(2, 0, 1).unsqueeze(0).float() / 255.0 for img in images]
# Define a sequence of transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # resize to 256x256
    transforms.ToTensor(),  # Convert to tensor
])

# Apply the transformations
img_tensor_ls = []
for img in images:
    img_tensor_ls.append(transform(img))

img_tensor = torch.stack(img_tensor_ls, dim=0) # stack the images along the batch dimension
# send images to GPU, if available
img_tensor = img_tensor.to(DEVICE)


# for each image, run synthesis with full PortillaSimoncelli model
model = po.simul.PortillaSimoncelli(img_tensor.shape[-2:]).to(DEVICE)
metamers = []
for i in range(img_tensor.shape[0]-4): # only run synthesis for the first, 5 images in the batch
    # metamer = run_synthesis(img_tensor[i:i+1], model)
    metamer = run_synthesis(img_tensor[i:i+1], model,loss_function=po.tools.optim.l2_norm)
    metamers.append(metamer)


In [ ]:
# print all images in img_tensor
po.imshow(img_tensor, as_rgb=True)
img_tensor.min(), img_tensor.max()


In [ ]:
# print the metamer images 
po.imshow([metamer.image for metamer in metamers], title='Target images', as_rgb=True)
# print target image
po.imshow([metamer.metamer for metamer in metamers], title='Synthesized metamer images', as_rgb=True)


Clearly, the above metamers show some discrepancies to the target image, some more, some less. While images 2, 3, and 4 approximate the texture of the target image quite well, the colors are a bit off, especially in image 4. Metamers 1 and 5 are off in both color and texture. It seems that homegeneous, finer patterns are well represented by the model, while distinct coarser structures, such as the arteries in the leaves or the cracks in the wood, are captured, but not in the same location. The result is a combined texture that doesn't reflect the original in the physical world. 

So let's, first tweak the model so that it performs better in terms of color in the metamers.

#### 2. Adapting the Model to Produce Better Color Metamers
**Combining Channel Information in RGB-Images**
- The current implementation of the Portilla-Simoncelli Model computes the statistics on all channels separately, i.e., treats each channel as a separate image, resulting in unrealistic metamers (see demo below).
- This is why we will play around with color transforms in the forward method.
- We will first mmic what [Brown 2023](https://dl.acm.org/doi/full/10.1145/3564605#sec-supp) et al. did.
- In particular, we will write our custom Portilla-Simoncelli class, where we do the following steps:

##### Implementation:
0. **Transform to LMS**:
    - transform RGB to an LMS-like space, then from that into opposing color channels (define rgb2opc)
    - Most of the color transforms are copied from color_utils.py in [PooledStatisticsMetamers](https://github.com/ProgramofComputerGraphics/PooledStatisticsMetamers/blob/main/poolstatmetamer/color_utils.py#L58) repo 
1. **Channel Separation**: The input image is split into its constituent channels.
    - compute L2 loss separately on each channel, the use torch.logsumexp to comine them
2. **Independent Processing**: Each channel is processed independently to compute the relevant statistics.
    - then add cross-channel correlations as in Brown
3. **Normalization and Weighting**: The computed statistics are normalized and weighted to ensure they contribute appropriately to the final representation. E.g., the green channel might be assigned higher weights as it  is often more sensitive in human vision.
4. **Fusion and Integration**: The normalized and weighted statistics are combined to form the final multi-channel representation.

In [ ]:
# let's first take care of the transforms
# A approximate transform from a typical RGB space to cone LMS space
# Would be good to check this to see if is a reasonable default approximation for 
# human cone LMS, but results so far don't seem to depend much on precise color transform details
# code largely copied from color_utils in PooledStatisticsMetamers repo 
rgb2lms = torch.tensor([[0.3811, 0.5783, 0.0402],
                        [0.1967, 0.7244, 0.0782],
                        [0.0241, 0.1288, 0.8444]])
lms2rgb = torch.inverse(rgb2lms)
# A simple approximation of the opponent cone color space (achromatic,red-green,blue-yellow)
# I also scaled their magnitudes to make the channels have more simlar ranges
lms2opc = torch.tensor([[0.5, 0.5, 0],    # (L+M) / 2)
                       [-4, 4, 0],        # (M-L) * 3
                       [0.5, 0.5, -1]])  # (L+M)/2 - S)

opc2lms = lms2opc.inverse()

# Composite transform from RGB to cone-opponent color space
rgb2opc = torch.matmul(lms2opc,rgb2lms)

opc2rgb = rgb2opc.inverse()

# Short names for the three opponent channels, short for (achromatic,red-green,blue-yellow)
opc_short_names = ('ac','rg','by')


# Apply the specified color tranformation matrix to an image
def color_transform_image(image,color_matrix):
    if image.dim()==3:
        return torch.nn.functional.conv2d(image[None,:,:,:],color_matrix[:,:,None,None])
    else:
        return torch.nn.functional.conv2d(image,color_matrix[:,:,None,None])

    
# Convert an RGB image to cone LMS image
def rgb_to_coneLMS(image):
    return color_transform_image(image,rgb2lms)

# Convert an RGB image to a cone opponent space image
def rgb_to_opponentcone(image):
    return color_transform_image(image,rgb2opc)


In [ ]:

# Single wood image
wood_img = img_tensor[0].unsqueeze(dim=0).to('cpu')
opc_image = einops.einsum(wood_img, rgb2opc, 'b c1 h w, c2 c1 -> b c2 h w') # tensor product of image and color matrix
# transform opc image to rgb
opc_image_retransf = einops.einsum(opc_image, opc2rgb, 'b c1 h w, c2 c1 -> b c2 h w')
po.imshow(opc_image, title='Cone opponent image', as_rgb=True)
po.imshow(opc_image_retransf, title='Cone opponent image retransformed to RGB', as_rgb=True)

In [ ]:
# Check if CUDA is available and move tensors to GPU if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move img_tensor and rgb2opc to the chosen device
img_tensor = img_tensor.to(device)
rgb2opc = rgb2opc.to(device)

# Convert the images to cone opponent space
img_opc_tensor = color_transform_image(img_tensor, rgb2opc)
po.imshow(img_opc_tensor, title='Cone opponent images', as_rgb=True)


In [ ]:
def rescale(image: torch.Tensor, range: tuple = (0, 1)):
    # Compute the min and max values across height and width for each image in the batch
    min_val = image.amin(dim=(2, 3), keepdim=True)
    max_val = image.amax(dim=(2, 3), keepdim=True)
    
    # Rescale the images to the range [0, 1]
    image = (image - min_val) / (max_val - min_val + 1e-8)  # Adding epsilon to avoid division by zero
    
    return image, min_val, max_val
def inverse_rescale(image: torch.Tensor, min_val: torch.Tensor, max_val: torch.Tensor):
    # Reverse the rescaling to original values
    image = image * (max_val - min_val) + min_val
    return image

# Compute rescaled images and min/max values
img_opc_tensor_rescaled, min_val, max_val = rescale(img_opc_tensor)
img_opc_tensor_resc_inv = inverse_rescale(img_opc_tensor_rescaled, min_val, max_val)
# Sanity check: the original and rescaled images should be the same
# Check if the inverse rescaling works correctly
torch.allclose(img_opc_tensor, img_opc_tensor_resc_inv, atol=1e-6)


In [ ]:
# show the rescaled opc images
po.imshow(img_opc_tensor_rescaled, title='Rescaled cone opponent images', as_rgb=True)
# show the original opc images
po.imshow(img_opc_tensor, title='Original cone opponent images', as_rgb=True)
# Sanity check: the original and rescaled images should be the same
# show the inverse rescaled opc images
#po.imshow(img_opc_tensor_resc_inv, title='Inverse rescaled cone opponent images', as_rgb=True)

In [ ]:

# single wood image:
# rescale the image to the range [0, 1] on final two dimensions
# opc_img_rescaled, min_val, max_val = rescale(opc_image) # single wood image
# po.imshow(opc_img_rescaled, title='Cone opponent image rescaled', as_rgb=True) # this is where we conduct the metamer synthesis on
# opc_img_inverse_rescaled = inverse_rescale(opc_img_rescaled, min_val, max_val)
# po.imshow(opc_img_inverse_rescaled, title='Cone opponent image inverse rescaled', as_rgb=True)

In [ ]:

print(f"Min and max values of the ORIGINAL images: {img_tensor.amin(dim=(2, 3)).squeeze()} and {img_tensor.amax(dim=(2, 3)).squeeze()}")
print(f"Min and max values of the OPC images: {img_opc_tensor.amin(dim=(2, 3)).squeeze()} and {img_opc_tensor.amax(dim=(2, 3)).squeeze()}")


In [ ]:
# plot histogram for each image in the batch of rescaled images
# def plot_histograms(images: torch.Tensor,  title: str,bins: int = 100):
#     # Create a figure and axis
#     fig, ax = plt.subplots(1, images.shape[0], figsize=(15, 5))
#     fig.suptitle(title)
    
#     # Plot the histograms for each image in the batch
#     for i in range(images.shape[0]):
#         ax[i].hist(images[i].flatten().cpu().numpy(), bins=bins)
#         ax[i].set_title(f'Image {i + 1}')
#         ax[i].set_xlabel('Pixel Intensity')
#         ax[i].set_ylabel('Frequency')
    
#     plt.show()


# plot histogram for each image in the batch of rescaled images
def plot_histograms(images: torch.Tensor, title: str, bins: int = 100):
    # Ensure images are on CPU and converted to numpy arrays
    images_np = images.cpu().numpy()

    # Create a figure and axis
    fig, axs = plt.subplots(images.shape[0], images.shape[1], figsize=(21, 9), sharex=True)
    fig.suptitle(title)
    
    # Plot the histograms for each channel of each image in the batch
    for i in range(images.shape[0]):  # iterate over images
        for j in range(images.shape[1]):  # iterate over channels
            axs[i, j].hist(images_np[i, j].flatten(), bins=bins)
            axs[i, j].set_title(f'Image {i + 1}, Channel {j + 1}')
            axs[i, j].set_xlabel('Pixel Intensity')
            axs[i, j].set_ylabel('Frequency')
    # plt.tight_layout()
    plt.show()

# histograms of the rgb images 
plot_histograms(img_tensor, title='Histograms of RGB Images')
#  histograms of the opc images prio to rescaling
plot_histograms(img_opc_tensor, title='Histograms of Cone Opponent Images')
#  histograms of the rescaled images
plot_histograms(img_opc_tensor_rescaled, title='Histograms of Rescaled Cone Opponent Images')
# print the min and max value for each image in the batch of rescaled images, opc images and rgb images


### custom Loss function
- compute the L2 norm on each channel separately 
- then combine channel losses with logsumexp

In [ ]:
def l2_channelwise(synth_rep: torch.Tensor, ref_rep: torch.Tensor, **kwargs) -> torch.Tensor:
    r"""l2-norm of the difference between ref_rep and synth_rep per channel.

    Parameters
    ----------
    synth_rep
        The first tensor to compare, model representation of the
        synthesized image.
    ref_rep
        The second tensor to compare, model representation of the
        reference image. must be same size as ``synth_rep``.
    kwargs
        Ignored, only present to absorb extra arguments.

    Returns
    -------
    loss
        The L2-norm of the difference between ``ref_rep`` and ``synth_rep``.

    """
    channel_losses = torch.linalg.vector_norm(ref_rep - synth_rep, dim=2,ord=2)
    # print(f'channel losses {channel_losses}, channel loss shape: {channel_losses.shape}')
    return torch.logsumexp(channel_losses, dim=1)

In [ ]:
#  The following class tweaks the PortillaSimoncelli model so that it will process color images better.
#  In particular, we introduce cross-color channel statistics, to capture the relationship between different color channels.
#  The cross-color statistics are a reduced set of statistis, currently only encompassing covariance/correlation. 
from typing import List, Optional, Any
from collections import OrderedDict

SCALES_TYPE = Optional[List[int]]

#from collections import OrderedDict
class PortillaSimoncelliCrossChannel(po.simul.PortillaSimoncelli):
    r"""Model for measuring a subset of texture statistics reported by PortillaSimoncelli

    Parameters
    ----------
    im_shape: int
        the size of the images being processed by the model, should be divisible by 2^n_scales
    remove_keys: list
        The dictionary keys for the statistics we will "remove".  In practice we set them to zero.
        Possible keys: ["pixel_statistics", "auto_correlation_magnitude",
        "skew_reconstructed", "kurtosis_reconstructed", "auto_correlation_reconstructed", 
        "std_reconstructed", "magnitude_std", "cross_orientation_correlation_magnitude", 
        "cross_scale_correlation_magnitude" "cross_scale_correlation_real", "var_highpass_residual"]
    """
    def __init__(
        self,
        im_shape,
    ):
        super().__init__(im_shape, n_scales=4, n_orientations=4, spatial_corr_width=9)
        
    def forward(self, image: torch.Tensor, scales: Optional[List[SCALES_TYPE]] = None) -> torch.Tensor:
        """
        Generate Texture Statistics representation of an image with cross-channel statistics.

        Parameters
        ----------
        image : Tensor
            A 4d tensor (batch, channel, height, width) containing the image(s) to analyze.
        scales : List[SCALES_TYPE], optional
            Which scales to include in the returned representation. If None, include all scales.

        Returns
        -------
        representation_tensor : Tensor
            3d tensor of shape (batch, channel, stats) containing the measured texture statistics.

        Raises
        ------
        ValueError
            If `image` is not 4d or has a dtype other than float or complex.
        """

        # call the parent class forward method to compute the base statistics
        base_representations = super().forward(image, scales=scales)

        # compute the cross-channel statistics
        cross_channel_stats = self._compute_cross_channel_stats(image)

        # modidfy the base representation to include the cross-channel statistics
        representation_tensor = torch.cat((base_representations, cross_channel_stats), dim=-1)

        # unnormalize representation_tensor
        # representation_tensor = representation_tensor * (max_val - min_val) + min_val

        return representation_tensor

    def _compute_cross_channel_stats(self, image:torch.Tensor) -> torch.Tensor:
        """
        Compute cross-channel statistics for the input image.

        Parameters
        ----------
        image : torch.Tensor
            A 4d tensor (batch, channel, height, width) containing the image(s) to analyze.

        Returns
        -------
        torch.Tensor
            3d tensor of shape (batch, channel, stats) containing the cross-channel statistics.
        """

        ## compute the cross-channel statistics
        cross_channel_stats = self._compute_cross_channel_covariance(image)



        return cross_channel_stats
    
    def _compute_cross_channel_covariance(self, image:torch.Tensor) -> torch.Tensor:
        """
        Compute the cross-channel covariance for the input image.

        Parameters
        ----------
        image : torch.Tensor
            A 4d tensor (batch, channel, height, width) containing the image(s), potentially in some preprocessed state like cone LMS, OPC space.

        Returns
        -------
        torch.Tensor
            3d tensor of shape (batch, channel, stats) containing the cross-channel covariance statistics.
        """

        # Compute the mean across the channel dimension
        mean_across_channels = image.mean(dim=(2, 3), keepdim=True)  # shape: [batch_size, num_channels, 1, 1]

        # Centering the data
        centered_data = image - mean_across_channels

        # Vectorized computation of covariance matrix
        covariance_matrix = (centered_data[:, :, None, :, :] * centered_data[:, None, :, :, :]).mean(dim=(3, 4))

        #print(f'covariance matrix requires grad: {covariance_matrix.requires_grad}')

        return 1e5*covariance_matrix
        
    # overwriting the following two methods allows us to use the plot_representation method
    # with the modified model, making examining it easier. In particular, it will add the new cross-channel statistics to the plot.
    def convert_to_dict(self, representation_tensor: torch.Tensor) -> OrderedDict:
        """Convert tensor of stats to dictionary."""
        #n_cross_channel_cov = self.n_scales * self.n_orientations
        n_cross_channel_cov = self.num_channels ** 2
        rep = super().convert_to_dict(representation_tensor[..., :-n_cross_channel_cov])
        cross_channel_cov = representation_tensor[..., -n_cross_channel_cov:]
        rep['cross_channel_covariance'] = einops.rearrange(cross_channel_cov, 'b c (s o) -> b c s o', s=self.n_scales, o=self.n_orientations)
        return rep

    def _representation_for_plotting(self, rep: OrderedDict) -> OrderedDict:
        r"""Convert the data into a dictionary representation that is more convenient for plotting.

        Intended as a helper function for plot_representation.
        """
        cross_channel_cov = rep.pop('cross_channel_covariance')
        data = super()._representation_for_plotting(rep)
        data['cross_channel_covariance'] = cross_channel_cov.flatten()
        return data


In [ ]:
# Assuming opc_images_rescaled is a batch of image tensors with shape [batch_size, channels, height, width]

# Create the model instance outside the loop
model = PortillaSimoncelliCrossChannel(img_opc_tensor_rescaled.shape[-2:]).to(DEVICE)

# Move all images to the desired device (assuming DEVICE is defined somewhere)
img_batch = img_opc_tensor_rescaled.to(DEVICE)

# number of target images in the batch
n_avail_imgs = img_batch.shape[0]
nth_img = n_avail_imgs # number of images in the batch
# or choose a number smaller than n_avail_imgs to accelaterate synthesis
#nth_img = 1


# Run synthesis on all images in batch
# metamers_opc = run_synthesis(img_batch, model, loss_function=l2_channelwise)
# initialize the an empty list to store the metamers
metamers_opc = []
for img in img_batch[0:nth_img]:
    print(img.shape)
    metamer = run_synthesis(img[None, :, :,:], model, loss_function=l2_channelwise)
    metamers_opc.append(metamer)

In [ ]:
#convert the metamers to a tensor
metamers_opc_tensor = torch.stack([metamer.metamer for metamer in metamers_opc], dim=0)
# drop the first dimension of the tensor
metamers_opc_tensor = metamers_opc_tensor.squeeze(1)
metamers_opc_tensor.shape

In [ ]:
# print opc metamer images and target images
po.imshow([metamer.image for metamer in metamers_opc], title='Target images, OPC, scaled', as_rgb=True)
po.imshow(metamers_opc_tensor, title='metamer images in OPC, scaled', as_rgb=True)

# inverse rescale metamer opc images
metamers_opc_tensor_inverse_rescaled = inverse_rescale(metamers_opc_tensor, min_val[:nth_img], max_val[:nth_img])
# compute the rgb metamer images
opc2rgb = opc2rgb.to(DEVICE)
metamers_rgb_tensor = color_transform_image(metamers_opc_tensor_inverse_rescaled, opc2rgb)
# print the rgb metamer images
po.imshow(metamers_rgb_tensor, title='metamer images in RGB', as_rgb=True)
# print the original image
po.imshow(img_tensor, title='Original Target images', as_rgb=True)

In [ ]:
# plot errors
[po.synth.metamer.plot_synthesis_status(metamer) for metamer in metamers_opc]
#po.synth.metamer.plot_synthesis_status(metamers_rgb_tensor)